In [1]:
from crewai import Agent, Task, Crew
import os
from dotenv import load_dotenv
import openai

In [2]:
# Carregar variáveis do .env
load_dotenv()

True

## Teste de multi-agentes guardando prompts anteriores

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew

load_dotenv()

# 1) O codigo tem dois agentes: COMPRADOR e VENDEDOR
comprador = Agent(
    role="Comprador de Camisas",
    goal="Comprar uma camisa branca para um casamento. Já tem o terno, só precisa da camisa.",
    backstory=(
        "Você é um cliente que quer economizar; procura algo bonito e não muito caro. "
        "Fale de forma breve, pergunte preços, negocie se possível."
    ),
    memory=False,
    verbose=False,
    allow_delegation=False,
    llm=os.getenv("OPENAI_MODEL_NAME") or "gpt-3.5-turbo"
)

vendedor = Agent(
    role="Vendedor de Loja de Roupas",
    goal="Vender a melhor camisa branca possível para o cliente.",
    backstory=(
        "Você conhece bem o estoque. Procure oferecer 3 opções, discuta preços e tente concretizar a venda."
    ),
    memory=False,
    verbose=False,
    allow_delegation=False,
    llm=os.getenv("OPENAI_MODEL_NAME") or "gpt-3.5-turbo"
)

# 2) O codigo vai manter um histórico de conversas, como texto puro
#    simulando um diálogo estilo:
#      "Comprador: ...\nVendedor: ...\nComprador: ...\n" assim os agentes quando recebem o histórico sabem quem falou o que
historico = """(Início da conversa)
Comprador: Olá, tudo bem? Preciso de uma camisa branca para um casamento.
"""

def gerar_proxima_fala(agent: Agent, conversa: str, quem_fala: str) -> str:
    """
    Cria uma Task para o 'agent' gerar a próxima fala no diálogo,
    recebendo como contexto todo o 'conversa' (histórico) e
    instruindo para falar como 'quem_fala'.
    Retorna apenas o texto da fala (sem prefixos).
    """
    # Monta o prompt de forma simples:
    prompt = f"""
Este é o histórico da conversa até agora:
----------------
{conversa}
----------------

Agora é a vez de {quem_fala} falar.
Responda como se você fosse {quem_fala} no diálogo.
Não faça rodeios sobre a própria IA, apenas continue a conversa.
"""

    # Cria a Task
    t = Task(
        description=prompt,
        expected_output=f"A próxima fala do {quem_fala} no diálogo.",
        agent=agent
    )
    # Executa via um Crew temporário
    c = Crew(agents=[agent], tasks=[t], verbose=False)
    result = c.kickoff()
    return result.raw.strip()


def executar_dialogo():
    # Para demonstrar, vao ser feitas 6 trocas de falas (3 rodadas). 3 rodadas, cada 2 falas => total 6 falas
    global historico

    # Já começa com o comprador na conversa (definido acima).
    # Vamos alternar: Vendedor -> Comprador -> Vendedor -> Comprador ...
    for rodada in range(3):
        # 1) Vendedor fala
        fala_vendedor = gerar_proxima_fala(vendedor, historico, "Vendedor")
        historico += f"\nVendedor: {fala_vendedor}\n"

        # 2) Comprador fala
        fala_comprador = gerar_proxima_fala(comprador, historico, "Comprador")
        historico += f"\nComprador: {fala_comprador}\n"

    # imprimimos todo o diálogo
    print("=== Diálogo Gerado ===")
    print(historico)

if __name__ == "__main__":
    executar_dialogo()


Overriding of current TracerProvider is not allowed


Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


=== Diálogo Gerado ===
(Início da conversa)
Comprador: Olá, tudo bem? Preciso de uma camisa branca para um casamento.

Vendedor: Olá! Fico feliz em te ajudar a encontrar a camisa branca perfeita para o casamento. Tenho três ótimas opções que podem interessar:

1. **Camisa de Algodão Premium** - Esta camisa é feita 100% em algodão egípcio, com um toque macio e um caimento excelente. É ideal para ocasiões formais. O preço é R$ 199,90.

2. **Camisa Slim Fit** - Com um design moderno e corte ajustado, essa camisa é perfeita para quem busca um visual elegante e contemporâneo. O tecido é leve e respirável, ideal para o clima do casamento. Essa custa R$ 169,90.

3. **Camisa Branca Clássica com Detalhes** - Esta opção traz um toque especial com detalhes sutis nas mangas e colarinho, sem perder a sofisticação. É um pouco mais casual, mas ainda assim elegante. O preço é R$ 149,90.

Você gostaria de experimentar alguma dessas opções? Posso te ajudar a encontrar o tamanho certo!

Comprador: Olá! A

## Teste multi-agentes 2 com memory

In [8]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

# Carregar variáveis do .env
load_dotenv()

# Usar as variáveis carregadas
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4-turbo")  # Define um padrão caso não esteja no .env

# Configuração do modelo de IA
llm = ChatOpenAI(model_name=model_name, openai_api_key=api_key, temperature=0.7)

# Memória resumida para manter o contexto sem gastar muitos tokens
memory = ConversationSummaryMemory(llm=llm)

# Definição dos agentes
comprador = Agent(
    role="Comprador",
    goal="Encontrar e comprar uma camisa branca elegante para um casamento pelo melhor preço possível.",
    backstory="Você tem um casamento para ir neste fim de semana e precisa de uma camisa branca elegante. "
              "Você quer pagar um preço justo, mas está disposto a negociar se necessário.",
    llm=llm,
    memory=memory
)

vendedor = Agent(
    role="Vendedor de Loja de Roupas",
    goal="Vender a melhor camisa branca para casamento pelo preço mais alto possível, sem perder a venda.",
    backstory="Você trabalha em uma loja de roupas sofisticada e sabe que tem camisas brancas de alta qualidade. "
              "Seu objetivo é convencer o comprador de que sua camisa é a melhor opção para a ocasião.",
    llm=llm,
    memory=memory
)

# Definição da tarefa principal (negociação de compra)
task = Task(
    description="O comprador e o vendedor devem negociar a compra de uma camisa branca para um casamento até que um acordo seja fechado.",
    agent=comprador,  # Define o agente principal da tarefa
    agents=[comprador, vendedor],  # Lista de agentes envolvidos
    memory=memory,
    max_rounds=6,  # Evita conversas infinitas
    termination_conditions=["compra realizada", "negociação encerrada", "cliente desistiu"],
    expected_output="Um acordo final entre o comprador e o vendedor sobre a compra da camisa branca."
)

# Criar a equipe com os dois agentes
crew = Crew(
    agents=[comprador, vendedor],
    tasks=[task]
)

# Iniciar a conversa
resultado = crew.kickoff()
print(resultado)


Overriding of current TracerProvider is not allowed


**Comprador:** Olá! Estou procurando uma camisa branca elegante para um casamento que vou neste fim de semana. Você tem alguma opção disponível?

**Vendedor:** Olá! Sim, temos algumas camisas brancas elegantes. Esta aqui é uma das nossas melhores opções, feita de algodão 100% e custa R$ 200.

**Comprador:** Parece uma boa camisa, mas o preço está um pouco acima do que eu esperava. Eu estava pensando em algo em torno de R$ 150. Existe alguma possibilidade de negociação?

**Vendedor:** Entendo, mas essa camisa realmente vale o preço, considerando a qualidade do material e o design. Posso oferecer um desconto de R$ 20, então ficaria por R$ 180.

**Comprador:** Agradeço pela oferta, mas ainda está acima do que eu posso pagar. Se eu fechar negócio agora, você conseguiria me vender por R$ 160?

**Vendedor:** Hmmm, R$ 160 é um pouco abaixo do que eu gostaria de aceitar, mas posso fazer essa exceção se você prometer que irá nos recomendar a seus amigos.

**Comprador:** Com certeza, eu sempre r

## TESTE DO CREWAI

In [3]:
# Criar o agente vendedor de roupas
vendedor_roupas = Agent(
    role="Vendedor de Loja de Roupas",
    goal="Ajudar clientes a escolher roupas ideais para suas necessidades.",
    backstory=(
        "Você trabalha em uma loja de roupas há 5 anos e tem um vasto conhecimento sobre tendências de moda. "
        "Sua prioridade é ajudar os clientes a encontrarem roupas que combinem com seu estilo e orçamento."
    ),
    verbose=True,
    allow_delegation=False,
    llm=os.getenv("OPENAI_MODEL_NAME")  # Modelo da OpenAI definido no .env
)

In [4]:
# Criar uma tarefa para o agente
tarefa_venda = Task(
    description="Um cliente entrou na loja e está procurando uma roupa para um evento. "
                "Converse com ele, descubra suas preferências e sugira algumas opções de roupas.",
    agent=vendedor_roupas,
    expected_output="O vendedor deve sugerir pelo menos 3 opções de roupas baseadas nas preferências do cliente."
)

In [5]:
# Criar a Crew e executar a interação
crew_vendas = Crew(
    agents=[vendedor_roupas],
    tasks=[tarefa_venda]
)

Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\guoli\Documents\Insper\PFE\PFE-NTT\venv\Lib\site-packages\urllib3\connectionpool.py", line 464, in _make_request
    self._validate_conn(conn)
  File "c:\Users\guoli\Documents\Insper\PFE\PFE-NTT\venv\Lib\site-packages\urllib3\connectionpool.py", line 1093, in _validate_conn
    conn.connect()
  File "c:\Users\guoli\Documents\Insper\PFE\PFE-NTT\venv\Lib\site-packages\urllib3\connection.py", line 741, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\guoli\Documents\Insper\PFE\PFE-NTT\venv\Lib\site-packages\urllib3\connection.py", line 920, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
               ^^^^^^^^^^^^^^^^
  File "c:\Users\guoli\Documents\Insper\PFE\PFE-NTT\venv\Lib\site-packages\urllib3\util\ssl_.py", line 460, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_imp

In [6]:
# Rodar a interação
resultado = crew_vendas.kickoff()
print("\nResposta do Vendedor:\n", resultado)

# Agent: Vendedor de Loja de Roupas
## Task: Um cliente entrou na loja e está procurando uma roupa para um evento. Converse com ele, descubra suas preferências e sugira algumas opções de roupas.


# Agent: Vendedor de Loja de Roupas
## Final Answer: 
Olá! Bem-vindo à loja! Estou aqui para ajudar você a encontrar a roupa perfeita para o seu evento. Poderia me dizer um pouco mais sobre o tipo de evento que você vai? É uma festa formal, casual, um casamento ou talvez algo mais descontraído? Além disso, gostaria de saber se você tem alguma preferência de cor, estilo ou tecido.  

Enquanto você pensa sobre isso, aqui estão algumas sugestões que podem funcionar para diferentes tipos de eventos:  

1. **Vestido Midi Floral (casual/outdoor)**: Se o evento for durante o dia e mais descontraído, um vestido midi floral pode ser uma excelente escolha. Ele é leve e confortável, perfeito para um evento ao ar livre, e você pode combiná-lo com sandálias ou sapatilhas.  

2. **Terno Slim Fit (formal)**

In [7]:
# Criar o agente vendedor de roupas
vendedor_roupas = Agent(
    role="Vendedor de Loja de Roupas",
    goal="Ajudar clientes a escolher roupas ideais para suas necessidades.",
    backstory=(
        "Você trabalha em uma loja de roupas há 5 anos e tem um vasto conhecimento sobre tendências de moda. "
        "Sua prioridade é ajudar os clientes a encontrarem roupas que combinem com seu estilo e orçamento."
    ),
    # IMPORTANTE: para manter o contexto ao longo de várias interações
    memory=True,  
    verbose=True,
    allow_delegation=False,
    llm=os.getenv("OPENAI_MODEL_NAME")  # Modelo da OpenAI definido no .env
)

def conversa_dinamica(agent: Agent):
    print("Vendedor: Olá! Em que posso ajudar você hoje?\n")

    while True:
        mensagem_cliente = input("Você: ")
        if mensagem_cliente.lower() in ["sair", "exit", "quit"]:
            print("\nVendedor: Foi um prazer ajudar. Até mais!\n")
            break

        # Criamos uma Task que representa a "pergunta" do cliente
        tarefa = Task(
            description=f"Cliente diz: '{mensagem_cliente}'\n\nResponda como um vendedor de roupas:",
            expected_output="Fornecer uma resposta apropriada de vendedor.",
            agent=agent,
            # Se quiser, pode colocar tools=[...] aqui ou no agente
        )

        # Criamos um Crew com essa única task
        crew_temporario = Crew(
            agents=[agent],
            tasks=[tarefa],
            verbose=False  # Pode deixar True se quiser ver logs de debug
        )

        # Executamos a crew para obter a resposta do vendedor
        resultado = crew_temporario.kickoff()

        # O "resultado" final do crew será o output da última Task;
        # podemos imprimir a resposta diretamente:
        print(f"\nVendedor: {resultado.raw}\n")

# Executar o chat
if __name__ == "__main__":
    conversa_dinamica(vendedor_roupas)

Vendedor: Olá! Em que posso ajudar você hoje?


Vendedor: Foi um prazer ajudar. Até mais!

